In [1]:
!pip install tpot cloudpickle mlflow fosforio[snowflake]

  Using cached https://files.pythonhosted.org/packages/c0/9d/a5a5422e00c034e16c8ad2e0361cf26578a8fcbbc45c6d605d2decf72bc7/TPOT-0.12.2-py3-none-any.whl
     |████████████████████████████████| 26.3MB 5.8MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/48/5d/acf5905c36149bbaec41ccf7f2b68814647347b72075ac0b1fe3022fdc73/tqdm-4.66.5-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/12/f8/d6bc56d20c60b2b66e005dfed16fdf8c46dbe559d40896d06c448a65f134/scikit_learn-1.5.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ac/91/fdbd611bfa9a2c91001ccbbc32b3717d370551a98d083fd0a0bbfcb4b537/xgboost-2.1.1-py3-none-manylinux2014_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/91/29/df4b9b42f2be0b623cbd5e2140cafcaa2bef0759a00b7b70104dcfe2fb51/joblib-1.4.2-py3-none-any.whl
Processing /home/mosaic-ai/.cache/pip/wheels/3c/85/2b/2580190404636bfc63e8de3dff629c03bb795021e1983a6cc7/stopit

     |████████████████████████████████| 235kB 81.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d9/5a/e7c31adbe875f2abbb91bd84cf2dc52d792b5a01506781dbcf25c91daf11/six-1.16.0-py2.py3-none-any.whl
     |████████████████████████████████| 61kB 16.1MB/s eta 0:00:01
     |████████████████████████████████| 204kB 80.6MB/s eta 0:00:01
     |████████████████████████████████| 81kB 24.1MB/s eta 0:00:01
     |████████████████████████████████| 61kB 18.8MB/s eta 0:00:01
     |████████████████████████████████| 71kB 21.8MB/s eta 0:00:01
     |████████████████████████████████| 512kB 77.3MB/s eta 0:00:01
     |████████████████████████████████| 317kB 78.0MB/s eta 0:00:01
     |████████████████████████████████| 747kB 71.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 18.1MB/s eta 0:00:01
     |████████████████████████████████| 215kB 79.7MB/s eta 0:00:01
     |████████████████████████████████| 112kB 85.9MB/s eta 0:00:01
     |████████████████████████████████| 307kB 8

  Using cached https://files.pythonhosted.org/packages/1c/d5/c84e1a17bf61d4df64ca866a1c9a913874b4e9bdc131ec689a0ad013fb36/certifi-2024.7.4-py3-none-any.whl
     |████████████████████████████████| 112kB 86.5MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/e5/3e/741d8c82801c347547f8a2a06aa57dbb1992be9e948df2ea0eda2c8b79e8/idna-3.7-py3-none-any.whl
     |████████████████████████████████| 4.4MB 76.6MB/s eta 0:00:01
     |████████████████████████████████| 204kB 80.1MB/s eta 0:00:01
     |████████████████████████████████| 71kB 15.0MB/s eta 0:00:01
     |████████████████████████████████| 143kB 73.8MB/s eta 0:00:01
     |████████████████████████████████| 122kB 77.7MB/s eta 0:00:01
     |████████████████████████████████| 81kB 22.7MB/s eta 0:00:01
     |████████████████████████████████| 184kB 83.1MB/s eta 0:00:01
     |████████████████████████████████| 92kB 21.0MB/s eta 0:00:01
ERROR: openapi-schema-validator 0.6.2 has requirement jsonschema<5.0.0,>=4.19.1, but you'll have

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import os
import json
import cloudpickle
import numpy as np
import pandas as pd
from tpot import TPOTRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature
import mlflow.sklearn
from fosforio import get_local_dataframe, get_dataframe


def experiment(exp_details, tpot_config, generations, population_size, cv, random_state, verbosity):
    """
    exp_details: {
        name: exp_name
        id: exp_id
        description: exp_description
        dataset: exp_dataset
        target_column: exp_target_column
        algo_details: None
    }
    tpot_config: dict
    generations: 0.5
    population_size: 0.5
    cv: 5
    random_state: 42
    verbosity: 2
    """
    # Tracking URI set
    mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URL", "http://mlflow-server"))

    # Setting experiment name
    mlflow.set_experiment(exp_details.get("name", "sample_experiment"))

    # Adding description to the experiment
    tags = {'mlflow.note.content': exp_details.get("description", "sample_description")}

    # Reading input data
    try:
        if exp_details.get("dataset").split(".")[-1].lower() in ["csv", "tsv", "xlsx", "xls"]:
            # Read the input data file from /data mount attached to this notebook pod
            input_file = "/data/" + exp_details.get("dataset")
            data = get_local_dataframe(input_file)
        else:
            # Input data is a dataset
            data = get_dataframe(exp_details.get("dataset"))
            # Adding this to log input data in mlflow
            input_file = '/tmp/input_data.csv'
            data.to_csv(input_file, index=False)            
    except Exception as ex:
        print(f"Unable to read input dataset.\nError: {ex}")

    # Data Preprocessing: Validating and encoding the data if required and imputing null values.
    data = data.fillna(method='pad')  # Filling null values with the previous ones
    data = data.fillna(method='bfill')  # Filling null value with the next ones
    df_target, le_target, df_feature, le_dict_feature, oh_enc_feature, le_column_feature, oh_column_feature = encoding(
        data, exp_details.get("target_column"))

    # Split the data into training and test sets. (0.75, 0.25) split.
    train_x, test_x, train_y, test_y = train_test_split(df_feature, df_target, train_size=0.75, test_size=0.25)

    # Registering datasets with mlflow experiment run
    dataset = mlflow.data.from_pandas(data, source=input_file)

    for algo, hyperparam in exp_details.get("algo_details").items():
        if not hyperparam and type(hyperparam) != dict:
            hyperparam = TPOTRegressor.default_config_dict[algo]
        print(f"Starting experiment run for {algo} with hyperparam: {hyperparam}")
        # Adding individual algorithm and its hyperparam to tpot config along with preprocessors and selectors
        tpot_config_dict = {**tpot_config, **{algo: hyperparam}}
        with mlflow.start_run(tags=tags):
            pipeline_optimizer = TPOTRegressor(
                generations=generations,
                population_size=population_size,
                cv=cv,
                random_state=random_state,
                verbosity=verbosity,
                config_dict=tpot_config_dict
            )

            pipeline_optimizer.fit(train_x, train_y)

            predicted_qualities = pipeline_optimizer.predict(test_x)

            matrices = eval_metrics(test_y, predicted_qualities)

            print(f"{str(pipeline_optimizer.fitted_pipeline_)} pipeline best selected by TPOT")
            print(f"  score: {pipeline_optimizer.score(test_x, test_y)}")

            try:
                # logging hyper params for the best run/pipeline chosen
                for step, _ in pipeline_optimizer.fitted_pipeline_.named_steps.items():
                    # Checking if the step used in TPOT pipeline is present in TPOT default config dict.
                    # if yes, then log only the hyperparams which are present in TPOT default config and not all.
                    step_name = [s for s in TPOTRegressor.default_config_dict.keys() if step.lower() in s.lower()]
                    for k, v in pipeline_optimizer.fitted_pipeline_.named_steps[step].get_params().items():
                        if step_name:
                            if k in TPOTRegressor.default_config_dict.get(step_name[0]):
                                mlflow.log_param(str(step) + "_" + str(k), v)
                        else:
                            mlflow.log_param(str(step) + "_" + str(k), v)
            except Exception as ex:
                print(f"Exception occurred in logging params to mlflow.\nex: {ex}")

            # logging model metric
            for i in matrices:
                if matrices[i]:
                    mlflow.log_metric(i, matrices[i])
                    print(i, matrices[i])
            mlflow.log_metric("score", pipeline_optimizer.score(test_x, test_y))

            # Log input data to MLflow run artifact.
            mlflow.log_artifact(input_file)

            # Registering datasets with mlflow experiment run
            mlflow.log_input(dataset, context="input")

            # Set custom tags
            mlflow.set_tags({
                "template_id": os.getenv("template_id", "sample_template_id"),
                "notebook_name": os.getenv("notebook_name", "sample_notebook_name"),
                "algorithm": algo,
                "algo_details": exp_details.get("algo_details"),
                "tpot_selected_pipeline": str(pipeline_optimizer.fitted_pipeline_)
            })

            predictions = pipeline_optimizer.fitted_pipeline_.predict(train_x)
            signature = infer_signature(train_x, predictions)

            # Storing score function for the model
            score_and_dump_func("/tmp/scoring_func")
            mlflow.log_artifact("/tmp/scoring_func")

            # Register the model
            mlflow.sklearn.log_model(
                pipeline_optimizer.fitted_pipeline_, "model",
                registered_model_name=exp_details.get("name", "sample_experiment"), signature=signature,
                pip_requirements=['mlflow==2.10.0', 'sqlalchemy==1.3.5']
            )

            # Exporting the autogenerated code of tpot for best pipeline
            pipeline_optimizer.export(f'tpot_exported_{exp_details.get("name")}_{algo}.py')
            mlflow.log_artifact(f'tpot_exported_{exp_details.get("name")}_{algo}.py')


def try_or(fn):
    try:
        out = fn()
        return out
    except:
        return None


def eval_metrics(actual, pred):
    """
    :param
    actual
    pred
    :returns
    rmse, mae, r2
    """
    return {
        "rmse": try_or(lambda: np.sqrt(mean_squared_error(actual, pred))),
        "mae": try_or(lambda: mean_absolute_error(actual, pred)),
        "r2": try_or(lambda: r2_score(actual, pred))
    }


def score_and_dump_func(file_path):
    """
    :param
    file_path
    """

    def score_func(model, request):
        """
        :param
        model
        request
        :returns
        score_output
        """
        # Enter your custom score function here

        score_output = "Success"
        return score_output

    with open(file_path, "wb") as out:
        cloudpickle.dump(score_func, out)


def encoding(df, target_column):
    """
    Checking whether encoding required in target and feature datasets.
    If required, then encoding them with label and one hot encoding.
    :param:
    df: input dataframe
    target_column: target column
    :returns:
    df_target: target dataframe
    le_target: target label encoder object
    df_feature: feature dataframe
    le_dict_feature: dict of feature label encoder objects
    oh_enc_feature: feature one hot encoder object
    le_column_feature: list of feature label encoder columns
    oh_column_feature: list of feature one hot encoder columns
    """
    df_target = df[[target_column]]
    le_target = None
    # Target column validation and encoding
    if df.dtypes[target_column].name in ['object', 'bool']:
        print(f"target_column is of {df.dtypes[target_column].name} datatype, encoding required.")
        le_target = LabelEncoder()
        df_target[target_column] = pd.DataFrame(le_target.fit_transform(df_target[target_column].astype(str)))
        print(f"Target column label encoded {df_target[target_column]}, object: {le_target}")

    # Feature column validation and encoding
    df_feature = df.drop(target_column, axis=1)
    non_numeric_cols = df_feature.select_dtypes(include=['object', 'bool']).columns.tolist()
    le_dict_feature = {}
    le_column_feature = []
    oh_column_feature = []
    oh_enc_feature = None
    if len(non_numeric_cols) >= 1:
        print(f"{non_numeric_cols} columns are non numeric in feature dataset, encoding required.")
        for col in non_numeric_cols:
            if df_feature[col].nunique() >= 10:
                le_column_feature.append(col)
            else:
                oh_column_feature.append(col)

        print(f"Columns identified to be encoded with label encoder: {le_column_feature}\n"
              f"Columns identified to be encoded with one hot encoder: {oh_column_feature}")

        # columns to be label encoded
        if len(le_column_feature) == 0:
            df_feature = df_feature
        else:
            for col in le_column_feature:
                le_dict_feature[col] = LabelEncoder()
                df_feature[col] = le_dict_feature[col].fit_transform(df_feature[col].astype(str))
                print(f"{col} column label encoded {df_feature[col]}, object: {le_dict_feature[col]}")

        # columns to be one hot encoded
        if len(oh_column_feature) == 0:
            df_feature = df_feature
        else:
            unique_combinations = pd.get_dummies(df_feature[oh_column_feature])
            unique_combinations_list = unique_combinations.columns.tolist()
            oh_enc_feature = OneHotEncoder()
            oh_encoded_array = oh_enc_feature.fit_transform(df_feature[oh_column_feature]).toarray() if len(oh_column_feature) > 1 else oh_enc_feature.fit_transform(df_feature[oh_column_feature]).toarray()
            df_oh_enc = pd.DataFrame(oh_encoded_array, columns=unique_combinations_list)
            df_feature = df_feature.drop(columns=oh_column_feature)
            df_feature = df_feature.join(df_oh_enc)
            print(f"new one hot encoded df: {oh_encoded_array}\n"
                  f"one hot encoder object: {oh_enc_feature}\n")
        print(f"final feature df created: {df_feature}")
    return df_target, le_target, df_feature, le_dict_feature, oh_enc_feature, le_column_feature, oh_column_feature


# Adding Preprocessors and Selectors with tpot default hyperparameter tuning
preprocessors_selectors = [
    # Preprocessors
    "sklearn.preprocessing.Binarizer",
    "sklearn.decomposition.FastICA",
    "sklearn.cluster.FeatureAgglomeration",
    "sklearn.preprocessing.MaxAbsScaler",
    "sklearn.preprocessing.MinMaxScaler",
    "sklearn.preprocessing.Normalizer",
    "sklearn.kernel_approximation.Nystroem",
    "sklearn.decomposition.PCA",
    "sklearn.preprocessing.PolynomialFeatures",
    "sklearn.kernel_approximation.RBFSampler",
    "sklearn.preprocessing.RobustScaler",
    "sklearn.preprocessing.StandardScaler",
    "tpot.builtins.ZeroCount",
    "tpot.builtins.OneHotEncoder",
    # Selectors
    "sklearn.feature_selection.SelectFwe",
    "sklearn.feature_selection.SelectPercentile",
    "sklearn.feature_selection.VarianceThreshold",
    "sklearn.feature_selection.SelectFromModel"
]
tpot_config = {key: TPOTRegressor.default_config_dict[key] for key in preprocessors_selectors}

# Running Experiment with user configured params.
print(f"Starting Experiment Execution with the following params:\n{os.getenv('EXPERIMENT_DETAILS')}\n")
experiment(exp_details=json.loads(os.getenv("EXPERIMENT_DETAILS")), tpot_config=tpot_config, generations=5,
           population_size=20, cv=5, random_state=42, verbosity=2)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/conda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.9/site-packages/ipykernel/kern

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/opt/conda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.9/site-packages/traitlets/config/application.py", line 1043, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.9/site-packages/ipykernel/kern

AttributeError: _ARRAY_API not found

/tmp/pip_packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.
Starting Experiment Execution with the following params:
null



AttributeError: 'NoneType' object has no attribute 'get'